In [38]:
import image_transformation as it
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np

In [39]:
# testing on images in tests/ folder
list_of_test = ['00000022_000.png', '00000033_000.png', '00000091_000.png', '00000091_001.png', 
                '00000213_000.png', '00000231_000.png', '00000831_000.png', '00003894_000.png',]

In [40]:

def rotate_image(image: Image, angle: int) -> Image:
    """
    Rotate an image by a given angle
    :param image: PIL image
    :param angle: int
    :return: PIL image
    """
    return image.rotate(angle)

def change_brightness(image: Image, factor: float) -> Image:
    """
    Change the brightness of an image
    :param image: PIL image
    :param factor: float ]
    :return: PIL image
    """
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(factor)

def add_noise(image: Image, factor: float) -> Image:
    """
    Add noise to an image
    :param image: PIL image
    :param factor: float
    :return: PIL image
    """
    np_image = np.array(image)
    noise = np.random.normal(0, factor, np_image.shape)
    noisy_image = np_image + noise
    noisy_image_clipped = np.clip(noisy_image, 0, 255)
    return Image.fromarray(noisy_image_clipped.astype(np.uint8))


def add_blur(image: Image, factor: float) -> Image:
    """
    Add blur to an image
    :param image: PIL image
    :param factor: float
    :return: PIL image
    """
    return image.filter(ImageFilter.GaussianBlur(factor))

def transform_image(image: Image, rotation: int = 0, brightness: float = 1.0, noise: float = 0.0, blur: float = 0.0) -> Image:
    """
    Transform an image by applying rotation, brightness, saturation, noise and blur
    :param image: PIL image
    :param rotation: int
    :param brightness: float
    :param noise: float
    :param blur: float
    :return: PIL image
    """
    image = rotate_image(image, rotation)
    image = change_brightness(image, brightness)
    image = add_noise(image, noise)
    image = add_blur(image, blur)
    return image

In [41]:
def create_transformed_image(image_path: str, transformed_image_path: str, rotation: int = 0, brightness: float = 1.0, noise: float = 0.0, blur: float = 0.0) -> None:
    """
    Add a transformed image to the dataset
    :param image_path: str
    :param transformed_image_path: str
    optional parameters
    :param rotation: int
    :param brightness: float
    :param noise: float
    :param blur: float
    :return: None
    """
    image = Image.open(image_path)
    transformed_image = transform_image(image, rotation, brightness, noise, blur)
    transformed_image.save(transformed_image_path)

def add_transformed_image_to_csv(csv_path: str, image_index: str, transformed_image_index: str) -> None:
    """
    Add a transformed image to the dataset
    :param csv_path: str
    :param image_index: str
    :param transformed_image_index: str
    """
    
    df = pd.read_csv(csv_path)
    image_info = df[df['Image Index'] == image_index].copy()
    image_info['Image Index'] = transformed_image_index

    # add the new image to the end of the csv
    df = pd.concat([df, image_info], ignore_index=True)
    df.to_csv(csv_path, index=False)

def add_new_image(image_path: str, csv_path: str, rotation: int = 0, brightness: float = 1.0, noise: float = 0.0, blur: float = 0.0) -> None:
    """
    Add a new image to the dataset
    :param image_path: str
    :param csv_path: str
    oprional parameters
    :param rotation: int
    :param brightness: float
    :param noise: float
    :param blur: float
    :return: None
    """
    params_string = f"r{rotation}_b{brightness}_n{noise}_bl{blur}"

    splited_list = image_path.split('/')
    path, image_index = splited_list[:-1], splited_list[-1]
    path = '/'.join(path)
    
    transformed_image_path = f"{path}/{image_index[:-4]}_{params_string}_transformed.png"

    create_transformed_image(image_path, transformed_image_path, rotation, brightness, noise, blur)
    add_transformed_image_to_csv(csv_path, image_index, transformed_image_path)

In [64]:
import os

def remove_image(image_path: str) -> None:
    """
    Remove an image from the dataset without using it
    :param image_path: str
    :return: None
    """
    if os.path.exists(image_path):
        os.remove(image_path)
    else:
        raise FileNotFoundError(f"Image {image_path} not found")


def remove_transformed_data(csv_path: str) -> int:
    """
    Remove a transformed image from the dataset
    :param csv_path: str
    :return: number of transformed images remaining
    """

    df = pd.read_csv(csv_path)
    df_transformed = df[df['Image Index'].str.contains('_transformed.png')]

    for index, row in df_transformed.iterrows():
        transform_image_path = row['Image Index']
        try:
            remove_image(transform_image_path)
            df = df.drop(index)
            "ok"
        except Exception as e:
            print(f"Error removing image {transform_image_path}: {e}")

    df.to_csv(csv_path, index=False)
    
    return len(df[df['Image Index'].str.contains('_transformed.png')])

In [43]:
def test_rotate_image(image, angle=90):
    image = Image.open('tests/' + image)
    rotated_image = it.rotate_image(image, angle)
    assert rotated_image.size == image.size
    assert rotated_image.mode == image.mode

    rotated_image.save('tests/rotated_images/rotated_' + image.filename.split('/')[-1])

In [44]:
test_rotate_image(image = list_of_test[0], angle=90)
test_rotate_image(image = list_of_test[1], angle=180)
print('All tests passed')

All tests passed


In [45]:
def test_change_brightness(image, factor=1.5):
    image = Image.open('tests/' + image)
    brightened_image = change_brightness(image, factor)
    assert brightened_image.size == image.size
    assert brightened_image.mode == image.mode

    # save the new image in the folder tests/brightened_images
    brightened_image.save('tests/brightened_images/brightened_' + image.filename.split('/')[-1])

In [46]:
def test_noise(image, factor=50):
    image = Image.open('tests/' + image)
    noisy_image = add_noise(image, factor)
    assert noisy_image.size == image.size
    assert noisy_image.mode == image.mode

    # save the new image in the folder tests/noisy_images
    noisy_image.save('tests/noisy_images/noisy_' + image.filename.split('/')[-1])

In [47]:
def test_blur(image, factor=5):
    image = Image.open('tests/' + image)
    blurred_image = add_blur(image, factor)
    assert blurred_image.size == image.size
    assert blurred_image.mode == image.mode

    # save the new image in the folder tests/blurred_images
    blurred_image.save('tests/blurred_images/blurred_' + image.filename.split('/')[-1])

In [48]:
test_change_brightness(image = list_of_test[0], factor=1.5)
test_change_brightness(image= list_of_test[1], factor=0.5)
test_change_brightness(image = list_of_test[2], factor=0.1)
test_noise(image = list_of_test[2], factor=50)
test_noise(image = list_of_test[3], factor=100)
test_noise(image = list_of_test[4], factor=200)
test_blur(image = list_of_test[3], factor=5)
test_blur(image = list_of_test[0], factor=5)
test_blur(image = list_of_test[1], factor=10)
print('All tests passed')


All tests passed


In [72]:
import time
def test_add_new_image(image_path: str, csv_path: str, rotation: int = 0, brightness: float = 1.0, noise: float = 0.0, blur: float = 0.0) -> None:
    add_new_image(image_path, csv_path, rotation, brightness, noise, blur)
    df = pd.read_csv(csv_path)
    # wait for the file to be saved
    assert df[df['Image Index'].str.contains(image_path[:-3])].shape[0] > 0

def test_remove_transformed_data(csv_path: str) -> None:
    # remove the transformed images from the csv
    # check if the number of transformed images is 0
    assert remove_transformed_data(csv_path) == 0

In [77]:
test_add_new_image(image_path='tests/' + list_of_test[0], csv_path='tests/test.csv', rotation=90, brightness=1.5, noise=50, blur=5)
test_add_new_image(image_path='tests/' + list_of_test[1], csv_path='tests/test.csv', rotation=180, brightness=0.5, noise=100, blur=10)
test_add_new_image(image_path='tests/' + list_of_test[2], csv_path='tests/test.csv', rotation=270, brightness=0.1, noise=200, blur=15)
test_add_new_image(image_path='tests/' + list_of_test[3], csv_path='tests/test.csv', rotation=90, brightness=1.5, noise=50, blur=5)
test_add_new_image(image_path='tests/' + list_of_test[4], csv_path='tests/test.csv', rotation=180, brightness=0.5, noise=100, blur=10)
test_add_new_image(image_path='tests/' + list_of_test[5], csv_path='tests/test.csv', rotation=270, brightness=0.1, noise=200, blur=15)
test_add_new_image(image_path='tests/' + list_of_test[6], csv_path='tests/test.csv', rotation=90, brightness=1.5, noise=50, blur=5)
test_add_new_image(image_path='tests/' + list_of_test[7], csv_path='tests/test.csv', rotation=180, brightness=0.5, noise=100, blur=10)
test_add_new_image(image_path='tests/' + list_of_test[0], csv_path='tests/test.csv', rotation=270, brightness=0.1, noise=200, blur=15)
test_add_new_image(image_path='tests/' + list_of_test[1], csv_path='tests/test.csv', rotation=90, brightness=1.5, noise=50, blur=5)

In [78]:
test_remove_transformed_data(csv_path='tests/test.csv')